In [2]:
import pathlib
import shutil
import SimpleITK as sitk
import pandas as pd
from tqdm import tqdm, trange
from image_processing import load_image, convert_nifti_batch_to_dicom
from image_processing import ImageProcess
from image_processing import read_dicom_series
from image_processing.util import save_file_path_to_pandas_table

ImportError: cannot import name 'ImageProcess' from 'image_processing' (/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/medical_image_process/image_processing/__init__.py)

<Figure size 432x288 with 0 Axes>

### Step 1
Statistics of file paths in the destination directory
If you have the pandas table of paths, then skip this step 

In [2]:
origin_data_home = '/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data'
new_data_home = '/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data_in_dicoms'
save_file_path_to_pandas_table(origin_data_home, pathlib.Path(new_data_home).parent/'nifti_path_info.csv')
info_df = pd.read_csv(pathlib.Path(new_data_home).parent/'nifti_path_info.csv')
info_df['out_file_path'] = info_df['file_path'].apply(lambda x: x.replace('composite_data', 'composite_data_in_dicoms').replace('.nii.gz',''))
info_df.head()
info_df['out_file_path'][0]

=  File paths saved to /media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/nifti_path_info.csv  =


'/media/tx-deepocean/Data/workdir/wyd/2022/ZheJiang_BaChongJian/Dataset/composite_data_in_dicoms/1.5/宋照英/composite_1_3'

### Setp 2
Covert nifti files into dicoms

In [ ]:
def convert_nifti_batch_to_dicom(info_table, nifti_col, dicom_col, overwrite=True):
    for ind, row in tqdm(info_table.iterrows()):
        nii_file_path = pathlib.Path(row[nifti_col])
        dicom_dir_path = pathlib.Path(row[dicom_col])
        if not overwrite:
            if dicom_dir_path.exists():
                continue
        else:
            if dicom_dir_path.exists():
                shutil.rmtree(dicom_dir_path, ignore_errors=True)
        imgprs = ImageProcess(nii_file_path)
        imgprs.saveimg(savepath=dicom_dir_path, format='dcm', meta_data=False)

In [11]:
convert_nifti_batch_to_dicom(info_df, 'file_path', 'out_file_path', overwirte=False)

210it [08:43,  2.49s/it]


### Step 3
Check the information of the new dicoms series

In [5]:
example_nifti_path = info_df['file_path'][0]
example_dicoms_path = info_df['out_file_path'][0]
dicoms_path = pathlib.Path(example_dicoms_path)
reader = sitk.ImageSeriesReader()
dicomName = reader.GetGDCMSeriesFileNames(str(dicoms_path))
reader.SetFileNames(dicomName)
sitk_image = reader.Execute()

In [6]:
sitk_image.GetSize()

(512, 512, 306)

In [7]:
nii_sitk_image = load_image(example_nifti_path)
nii_sitk_image.GetSize()

(512, 512, 306)

In [8]:
sitk_image.GetSpacing()

(0.711000025272369, 0.711000025272369, 1.0)

In [9]:
nii_sitk_image.GetSpacing()

(0.7110000252723694, 0.7110000252723694, 1.0)

In [3]:
from pathlib import Path
Path('./sss.csv').suffix

'.csv'

In [16]:
path3 = Path('/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/medical_image_process/output/report_saved').parts[-3:]
path3

('medical_image_process', 'output', 'report_saved')

In [17]:
def create_path_from_names(path_names):
    path = Path(path_names[0])
    
    for name in path_names[1:]:
        path = path / name

    return path

In [20]:
part_path = create_path_from_names(path3)
part_path

PosixPath('medical_image_process/output/report_saved')

In [21]:
tt = Path('/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/')/part_path

In [22]:
tt

PosixPath('/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/medical_image_process/output/report_saved')

In [33]:
tt1 = tt.with_suffix('.csv')
tt1

PosixPath('/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/medical_image_process/output/report_saved.csv')

In [39]:
tt1 = tt1.with_suffix('.nii.gz')
tt1

PosixPath('/media/tx-deepocean/Data/workdir/wyd/2023/BeiDaRenMin/Osteosarcoma/medical_image_process/output/report_saved.nii.nii.gz')

In [31]:
import numpy as np
df1 = pd.DataFrame(np.ones((2,2)))
df1

,0,1
0,1.0,1.0
1,1.0,1.0


In [32]:
df1.iloc[0,:][0]

1.0

In [40]:
import pandas as pd
label_file = pd.read_excel('../Dataset/radiogeneomics_data.xlsx', sheet_name='114（化疗方案一致）')
label_file.head()

,lable,ID,sex,age,location,ALP,pathology type,size
0,0,82769550,0,7,1,0.0,1,11.0
1,0,82709979,1,23,1,1.0,1,7.5
2,0,82711996,0,12,1,0.0,2,15.1
3,0,82797007,1,5,1,0.0,1,5.3
4,0,82866515,0,8,1,1.0,1,7.1


In [41]:
label_file.rename(columns={'lable':'label'}, inplace=True)
label_file.to_csv('../Dataset/label.csv', index=0)